In [2]:
import numpy as np
import pandas as pd

In [3]:
data = [[1,0,0,1],[0,1,0,0],[0,1,0,1],[1,0,0,1],\
        [0,1,1,0],[1,0,1,0],[0,0,0,1],[0,1,1,0],[1,0,0,1],[1,1,0,1]]

In [4]:
Data = pd.DataFrame(data,columns=['x1','x2','x3','y'])
Data.head()

,x1,x2,x3,y
0,1,0,0,1
1,0,1,0,0
2,0,1,0,1
3,1,0,0,1
4,0,1,1,0


In [5]:
cols=Data.shape[1]
X_data=Data.iloc[:,:cols-1]
Y_data = Data.iloc[:,cols-1:]
featureNames=X_data.columns


In [6]:
def Naive_Bayes(X_data,Y_data):
    y=Y_data.values
    x=X_data.values
    y_unique=np.unique(y)
    prior_prob=np.zeros(len(y_unique))
    for i in range(len(y_unique)):
        prior_prob[i]=sum(y==y_unique[i]/len(y))
    condition_prob={}
    for feat in featureNames:
        x_unique=list(set(X_data[feat]))
        x_condition_prob=np.zeros((len(y_unique),len(x_unique)))
        for j in range(len(y_unique)):
            for k in range(len(x_unique)):
                x_condition_prob[j,k]=sum((X_data[feat]==x_unique[k])&(Y_data.y==y_unique[j]))/sum(y==y_unique[j])
        x_condition_prob = pd.DataFrame(x_condition_prob,columns=x_unique,index=y_unique)
        condition_prob[feat] = x_condition_prob
    return prior_prob,condition_prob


In [7]:
prior_prob,condition_prob=Naive_Bayes(X_data,Y_data)
print(prior_prob)
print(condition_prob['x1'])
print(condition_prob['x2'])
print(condition_prob['x3'])

[4. 0.]
          0         1
0  0.750000  0.250000
1  0.333333  0.666667
          0         1
0  0.250000  0.750000
1  0.666667  0.333333
      0     1
0  0.25  0.75
1  1.00  0.00


In [8]:
##testData is a dataFrame
def Prediction(testData,prior,condition_prob):
    numclass =prior.shape[0]
    featureNames =testData.columns
    numclass =prior.shape[0]
    numsample =testData.shape [0]
    featureNames =testData.columns
    post_prob =np.zeros((numsample,numclass))
    for k in range(numsample):
        prob_k =np.zeros((numclass,))
        for i in range(numclass):
            pri =prior[i]
            for feat in featureNames:
                feat_val =testData[feat][k]
                cp =condition_prob[feat]
                cp_val=cp.loc[i,feat_val]
                pri *=cp_val
            prob_k[i]=pri
        prob =prob_k/np.sum(prob_k,axis =0)
        post_prob[k,:]=prob
    return post_prob

In [9]:
test_data =[[0,0,1]]
testData =pd.DataFrame(test_data,columns=['x1','x2','x3'])#cons
testData.head()
postPrior =Prediction(testData,prior_prob,condition_prob)
postPrior

array([[1., 0.]])

 由运行结果可知没有拉普拉斯平滑时由于c属性为1时必为0类，因此直接判定为0类

### 以下是带有拉普拉斯平滑后的代码

In [10]:
def LNaive_Bayes(X_data, Y_data, laplace_smoothing=1):
    y = Y_data.values.flatten()  # 将Y_data转换为一维数组
    x = X_data.values
    y_unique = np.unique(y)
    prior_prob = np.zeros(len(y_unique))
    
    for i in range(len(y_unique)):
        prior_prob[i] = (sum(y == y_unique[i]) + laplace_smoothing) / (len(y) + len(y_unique) * laplace_smoothing)

    # Calculate conditional probabilities with Laplace smoothing
    condition_prob = {}
    for feat in X_data.columns:
        x_unique = list(set(X_data[feat]))
        p_x_condition_prob = np.zeros((len(y_unique), len(x_unique)))
        for j in range(len(y_unique)):
            for k in range(len(x_unique)):
                p_x_condition_prob[j, k] = (sum((X_data[feat] == x_unique[k]) & (y == y_unique[j])) + laplace_smoothing) / (sum(y == y_unique[j]) + len(x_unique) * laplace_smoothing)
        p_x_condition_prob = pd.DataFrame(p_x_condition_prob, columns=x_unique, index=y_unique)
        condition_prob[feat] = p_x_condition_prob

    return prior_prob, condition_prob

In [11]:
prior_prob,condition_prob=LNaive_Bayes(X_data,Y_data)
print(prior_prob)
print(condition_prob['x1'])
print(condition_prob['x2'])
print(condition_prob['x3'])

[0.41666667 0.58333333]
          0         1
0  0.666667  0.333333
1  0.375000  0.625000
          0         1
0  0.333333  0.666667
1  0.625000  0.375000
          0         1
0  0.333333  0.666667
1  0.875000  0.125000


In [12]:
##testData is a dataFrame
def Prediction(testData,prior,condition_prob):
    numclass =prior.shape[0]
    featureNames =testData.columns
    numclass =prior.shape[0]
    numsample =testData.shape [0]
    featureNames =testData.columns
    post_prob =np.zeros((numsample,numclass))
    for k in range(numsample):
        prob_k =np.zeros((numclass,))
        for i in range(numclass):
            pri =prior[i]
            for feat in featureNames:
                feat_val =testData[feat][k]
                cp =condition_prob[feat]
                cp_val=cp.loc[i,feat_val]
                pri *=cp_val
            prob_k[i]=pri
        prob =prob_k/np.sum(prob_k,axis =0)
        post_prob[k,:]=prob
    return post_prob

In [13]:
test_data =[[0,0,1]]
testData =pd.DataFrame(test_data,columns=['x1','x2','x3'])#cons
testData.head()
postPrior =Prediction(testData,prior_prob,condition_prob)
postPrior

array([[0.783174, 0.216826]])

拉普拉斯平滑之后的结果就并不那么一定

In [14]:
import numpy as np
import pandas as pd
data=[['Sunny','Hot','High','Weak','No'],['Sunny','Hot','High','Strong','No'],['Overcast','Hot','High','Weak','Yes'],\
      ['Rain','Mild','High','Weak','Yes'],['Rain','Cool','Normal','Weak','Yes'],['Rain','Cool','Normal','Strong','No'],\
      ['Overcast','Cool','Normal','Strong','Yes'],['Sunny','Mild','High','Weak','No'],['Sunny','Cool','Normal','Weak','Yes'],\
      ['Rain','Mild','Normal','Weak','Yes'],['Sunny','Mild','Normal','Strong','Yes'],['Overcast','Mild','High','Strong','Yes'],\
      ['Overcast','Hot','Normal','Weak','Yes'],['Rain','Mild','High','Strong','No']]
Data = pd.DataFrame(data, columns=['x1', 'x2', 'x3','x4', 'y'])
Data.head()

,x1,x2,x3,x4,y
0,Sunny,Hot,High,Weak,No
1,Sunny,Hot,High,Strong,No
2,Overcast,Hot,High,Weak,Yes
3,Rain,Mild,High,Weak,Yes
4,Rain,Cool,Normal,Weak,Yes


In [16]:
cols = Data.shape[1]
X_data = Data.iloc[:, :cols-1]
Y_data = Data.iloc[:, cols-1:]
featureNames = X_data.columns
def Naive_Bayes(X_data,Y_data):
    y = Y_data.values
    x = X_data.values
    y_unique = np.unique(y)
    prior_prob = np.zeros (len(y_unique))
    for i in range(len(y_unique)):
        prior_prob[i] = sum(y == y_unique[i]) / len(y)
        
    condition_prob = {}

    for feat in featureNames:
        x_unique = list(set(X_data[feat]))
        x_condition_prob = np.zeros((len(y_unique), len(x_unique)))
        for j in range (len(y_unique)):
            for k in range (len(x_unique)):
                x_condition_prob[j, k] = sum((X_data[feat] == x_unique[k]) & (Y_data.y == y_unique[j])) / sum(y == y_unique[j])
        x_condition_prob = pd.DataFrame(x_condition_prob, columns = x_unique, index = [0,1])
        condition_prob[feat] = x_condition_prob
    return prior_prob, condition_prob
def Prediction(testData, prior, condition_prob):
    numclass = prior.shape[0]
    featureName = testData.columns
    numsample = testData.shape[0]
    post_prob = np.zeros ((numsample, numclass))
    for k in range(numsample):
        prob_k = np.zeros((numclass,))
        for i in range(numclass):
            pri = prior[i]
            for feat in featureNames:
                feat_val = testData[feat][k]
                cp = condition_prob[feat]
                cp_val = cp.loc[i, feat_val]
                pri *= cp_val
            prob_k[i] = pri
        prob = prob_k / np.sum(prob_k, axis = 0)
        post_prob[k, :] = prob
    return post_prob
test_data = [['Sunny', 'Cool', 'High', 'Strong']]
testData = pd.DataFrame(test_data, columns = ['x1', 'x2', 'x3', 'x4'])
testData.head()
prior_prob, condition_prob = Naive_Bayes(X_data,Y_data)
postPrior = Prediction(testData, prior_prob, condition_prob)
postPrior
if (postPrior[0][0]>postPrior[0][1]):
    print('不去打球')

else:
    print('去打球')


不去打球


因此不去打球